In [32]:
%pip install boto3 'moto[all]' pytest ipytest

In [37]:
import boto3
from moto import mock_aws
import pytest
import ipytest

ipytest.autoconfig()
# --- Your model ---
class MyModel:
    def __init__(self, name, value):
        self.name = name
        self.value = value

    def save(self):
        s3 = boto3.client("s3", region_name="us-east-1")
        s3.put_object(Bucket="mybucket", Key=self.name, Body=self.value)

# --- Test using Moto ---
@mock_aws
def test_my_model_save():
    # Create a bucket in the in-memory Moto S3
    client = boto3.client("s3", region_name="us-east-1")
    client.create_bucket(Bucket="mybucket")

    # Instantiate your model and save
    model_instance = MyModel("steve", "is awesome")
    model_instance.save()

    # Retrieve the object from the mock S3
    body = client.get_object(Bucket="mybucket", Key="steve")["Body"].read().decode("utf-8")
    assert body == "is awesome"
    print("Test passed! Object in mock S3:", body)

# --- Run pytest manually in the notebook ---
if __name__ == "__main__":
    # This runs all pytest functions in this module
    ipytest.run()


.                                                                                            [100%]
1 passed in 1.81s


In [38]:
import unittest
import boto3
from moto import mock_aws

class TestMyModel(unittest.TestCase):

    @mock_aws
    def test_save(self):
        s3 = boto3.client("s3", region_name="us-east-1")
        s3.create_bucket(Bucket="mybucket")
        s3.put_object(Bucket="mybucket", Key="steve", Body="is awesome")

        body = s3.get_object(Bucket="mybucket", Key="steve")["Body"].read().decode()
        self.assertEqual(body, "is awesome")
        print("Test passed!")

# Run unittest manually in notebook
unittest.main(argv=[''], verbosity=2, exit=False)

test_save (__main__.TestMyModel.test_save) ... 

Test passed!


ok

----------------------------------------------------------------------
Ran 1 test in 0.869s

OK
